In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import ast
import pandas as pd
import processing as procdata

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
# SPARK
file_tweetspreproc = spark.read.csv('s3a://pi-g2-processado/spark-g2-tweets-preprocessado.csv', header=True).toPandas()
file_prounipreproc = spark.read.csv('s3a://pi-g2-processado/spark-g2-prouni-preprocessado.csv', header=True).toPandas()

In [2]:
# PANDAS
file_tweetspreproc = pd.read_csv('processed_data/g2-tweets-preprocessado.csv')
file_prounipreproc = pd.read_csv('processed_data/g2-prouni-preprocessado.csv')

In [3]:
dfprouni = pd.DataFrame({
    'curso': file_prounipreproc['curso'],
    'estado': file_prounipreproc['estado'],
    'mensalidade': file_prounipreproc['mensalidade'].map(lambda val: procdata.round_range(val, start=100, stop=15_000, step=500)),
    'quantidade_bolsas': file_prounipreproc['quantidade_bolsas'].map(lambda qtd: procdata.round_range(qtd, start=1, stop=500, step=2)),
    'nota_bolsa': file_prounipreproc['nota_bolsa'].map(lambda nota: procdata.round_range(nota, start=10, stop=1_000, step=10)),
    'sexo': file_prounipreproc['sexo'].map(lambda sexo: 'Masculino' if sexo == 'M' else 'Feminino'),
    'raca': file_prounipreproc['raca'].map(lambda raca: 'Outro' if raca not in ['Parda', 'Branca', 'Preta'] else raca),
    'deficiente': file_prounipreproc['deficiente'].map(lambda deficiente: 'Sim' if deficiente == 'S' else 'Não'),
    'idade': file_prounipreproc['idade'].map(lambda idade: procdata.round_range(procdata.get_age_from_birthdate(idade), start=16, stop=90, step=2)),
    'esgoto_inexistente': file_prounipreproc['esgoto_inexistente'].map(procdata.get_label_from_floatbool),
    'energia_inexistente': file_prounipreproc['energia_inexistente'].map(procdata.get_label_from_floatbool),
    'agua_inexistente': file_prounipreproc['agua_inexistente'].map(procdata.get_label_from_floatbool),
    'acesso_internet': file_prounipreproc['acesso_internet'].map(procdata.get_label_from_floatbool),
    'faz_exame_selecao': file_prounipreproc['faz_exame_selecao'].map(procdata.get_label_from_floatbool),
    'especializada_deficientes': file_prounipreproc['especializada_deficientes'].map(procdata.get_label_from_floatbool),
    'ensino_tecnico': file_prounipreproc['ensino_tecnico'].map(procdata.get_label_from_floatbool)
})
print(dfprouni.__len__(), "rows")
dfprouni.sample(n=5)

25856 rows


,curso,estado,mensalidade,quantidade_bolsas,nota_bolsa,sexo,raca,deficiente,idade,esgoto_inexistente,energia_inexistente,agua_inexistente,acesso_internet,faz_exame_selecao,especializada_deficientes,ensino_tecnico
13738,Pedagogia,SP,100,1,580,Feminino,Preta,Não,24,Não,Não,Não,Sim,Não,Sim,Não
23710,Pedagogia,SP,100,5,540,Feminino,Branca,Não,30,Não,Não,Não,Sim,Não,Não,Não
7454,Design Gráfico,SP,600,1,580,Feminino,Parda,Não,24,Não,Não,Não,Sim,Não,Sim,Não
25825,Sociologia,SP,100,1,530,Masculino,Preta,Não,34,Não,Não,Não,Sim,Não,Sim,Não
5060,Engenharia Mecânica,PB,600,49,450,Masculino,Parda,Não,26,Não,Não,Não,Sim,Não,Não,Não


In [4]:
file_tweetspreproc["date"] = file_tweetspreproc["date"].map(procdata.get_date)
file_tweetspreproc["frequent"] = file_tweetspreproc["frequent"].map(ast.literal_eval)

In [5]:
tweets = file_tweetspreproc['frequent'].agg('sum')
twtts = procdata.freq_tweet(tweets)

print("Sem tratamento", len(twtts))

tweetwordfreq = pd.DataFrame(twtts, columns=['palavra', 'frequencia'])\
    .drop_duplicates(subset=['palavra'])\
    .sort_values(by=['frequencia'], ascending=False)

print("Com tratamento", tweetwordfreq.__len__())

tempdftweets = file_tweetspreproc.copy()
tempdftweets['frequent'] = procdata.words_in_tweet(tempdftweets['frequent'])

tweetwordfreq['sentimento'] = tweetwordfreq['palavra'].apply(lambda w: procdata.get_sentiment_of_value(tempdftweets, w))
tweetwordfreq = tweetwordfreq.reset_index(drop=True)
tweetwordfreq.head()

Sem tratamento 1835
Com tratamento 951


,palavra,frequencia,sentimento
0,prouni,150,Negativo
1,q,45,Negativo
2,pra,39,Negativo
3,faculdade,27,Negativo
4,ter,23,Positivo


In [6]:
dftweetspreproc = file_tweetspreproc.drop(file_tweetspreproc[file_tweetspreproc["sentiment"] == 0].index)
sdates = file_tweetspreproc['date'].map(lambda x: x.date()).drop_duplicates().reset_index(drop=True)
ssentdate = sdates.apply(lambda sd: dftweetspreproc.loc[dftweetspreproc['date'].apply(lambda d: d.date() == sd)]['sentiment'].mean())
threebest = sdates.apply(lambda sd: file_tweetspreproc.loc[file_tweetspreproc['date'].apply(lambda d: d.date() == sd)]['frequent'].agg('sum'))
tweetsdayfreq = pd.DataFrame({
    'data': sdates,
    'sentimento': ssentdate,
    'palavra': threebest.map(procdata.freq_tweet_day)
})
tweetsdayfreq

,data,sentimento,palavra
0,2022-05-24,0.027600,"passei, terceira, chamada"
1,2022-05-23,-0.170030,"papel, fies, prouni"
2,2022-05-22,-0.249757,"23, anos, 4"
3,2022-05-21,0.151621,"ensino, médio, pra"
4,2022-05-20,-0.111908,"so, qria, saber"
5,2022-05-19,-0.204446,"vamo, anuncia, logo"
6,2022-05-18,-0.035032,"impressionante, alunos, prouni"
7,2022-05-17,-0.083629,"rede, diria, né"


In [ ]:
# SPARK
spark.createDataFrame(dfprouni).write.csv('s3a://pi-g2-cliente/spark-prouni.csv', header=True)
spark.createDataFrame(tweetwordfreq).write.csv('s3a://pi-g2-cliente/spark-tweets-wordfreq.csv', header=True)
spark.createDataFrame(tweetsdayfreq).write.csv('s3a://pi-g2-cliente/spark-tweets-worddaysfreq.csv', header=True)

In [7]:
# PANDAS
dfprouni.to_csv("client_data/prouni.csv")
tweetwordfreq.to_csv("client_data/tweets-wordfreq.csv")
tweetsdayfreq.to_csv("client_data/tweets-worddaysfreq.csv")